pouzite kniznice so far:
pandas ,scikit-learn

Task 1 - Data analysis:
(potrobnejšie info o data sete máme z https://www.kaggle.com/datasets/uciml/faulty-steel-plates )
Prvne sme zanalizovali veľkosť našeho data setu. Má 1941 položiek pričom každá položka má 34 stĺpcov.

Pokiaľ si zobereme čo aký stĺpec znamená, tak prvých 27 mi určuje parametre položky (v našom prípade sa jedná o Oceľové Pláty). Parametre sú celočíeslne (14 stĺpcov) alebo reálne (13. stĺpcov) hodnoty. Niektoré tieto parametre sú napríklad:

X_Minimum
X_Maximum
Y_Minimum
Y_Maximum
Pixels_Areas
X_Perimeter
Y_Perimeter
Sum_of_Luminosity
Minimum_of_Luminosity ...

Posledných 7 stĺpcov nám určuje či daná položka ma nejakú vadu. Prvých 6 je nejaká konkrétna vada a posledná sú nespecifikované vady. Tieto stĺpce máju boolovsku hodnotu (čiže 0 alebo 1). Pričom 0 znamená, že daná položka nemá túto vadu a 1 zase znamená, že túto vadu obsahuje. Možné vady sú:

    Pastry
    Z_Scratch
    K_Scatch
    Stains
    Dirtiness
    Bumps
    Other_Faults

Data set neobsahuje žiadne chýbajúce hodnoty.

In [1]:
import pandas as pd

df = pd.read_csv("data_set_basic.csv")
print(df.shape)

df.info()
print("Mame chybajuce hodnoty?")
print(df.isna().sum().sum())
print("Mame duplikaty?")
print(df.duplicated().sum())
print(df.columns.tolist())

(1941, 34)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 34 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      1941 non-null   int64  
 1   V2      1941 non-null   int64  
 2   V3      1941 non-null   int64  
 3   V4      1941 non-null   int64  
 4   V5      1941 non-null   int64  
 5   V6      1941 non-null   int64  
 6   V7      1941 non-null   int64  
 7   V8      1941 non-null   int64  
 8   V9      1941 non-null   int64  
 9   V10     1941 non-null   int64  
 10  V11     1941 non-null   int64  
 11  V12     1941 non-null   int64  
 12  V13     1941 non-null   int64  
 13  V14     1941 non-null   int64  
 14  V15     1941 non-null   float64
 15  V16     1941 non-null   float64
 16  V17     1941 non-null   float64
 17  V18     1941 non-null   float64
 18  V19     1941 non-null   float64
 19  V20     1941 non-null   float64
 20  V21     1941 non-null   float64
 21  V22     1941 non-null   fl

Data preprocessing, Normalization and Data Split

Keďže nemáme chýbajúce hodnoty ani duplikátne hodnoty môžeme ísť rovno k ďaľším krokom.

Rozdelíme data set na 2 polovice, X (vstupy, features) a Y (očakavaný výstup).

In [2]:

import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("data_set_basic.csv")

# Fault columns (the 7 binary target columns)
fault_columns = ['V28','V29','V30','V31','V32','V33','Class']


df['target'] = df[fault_columns].idxmax(axis=1)

# Oddelenie vstupov a výstupov
X = df.drop(columns=fault_columns + ['target'])
y = df['target']

# Enkodujeme
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

#Normalizácia
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nNormalizovane features:")
print(pd.DataFrame(X_scaled).describe().T.head(10))

# Shape summary
print(f"\nX shape: {X_scaled.shape}")
print(f"y shape: {y_encoded.shape}")

# Najprv rozdelime na train (70 %) a docasny zvysok (30 %)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y_encoded,
    test_size=0.3,          # 30 % pre val + test
    stratify=y_encoded,     # zachova rovnovahu tried
    random_state=42
)

# Rozdelime zvysok (30 %) na validation (15 %) a test (15 %)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,          # polovica z 30 % = 15 %
    stratify=y_temp,
    random_state=42
)

# Overenie rozmerov
print("\n--- Rozdelenie datasetu ---")
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Testovaci set: {X_test.shape}, {y_test.shape}")



Normalizovane features:
    count          mean       std       min       25%       50%       75%  \
0  1941.0  5.857127e-17  1.000258 -1.097164 -0.999192 -0.261520  0.925671   
1  1941.0 -8.785691e-17  1.000258 -1.234101 -0.856211 -0.303447  0.912636   
2  1941.0  1.464282e-17  1.000258 -0.926641 -0.664798 -0.251706  0.300086   
3  1941.0 -1.830352e-17  1.000258 -0.926658 -0.664808 -0.251730  0.300059   
4  1941.0  0.000000e+00  1.000258 -0.366137 -0.350268 -0.332850 -0.207442   
5  1941.0  2.928564e-17  1.000258 -0.364808 -0.321638 -0.285109 -0.092502   
6  1941.0  1.464282e-17  1.000258 -0.192240 -0.164096 -0.135951  0.000080   
7  1941.0  2.928564e-17  1.000258 -0.402338 -0.384234 -0.365334 -0.240747   
8  1941.0 -5.857127e-17  1.000258 -2.631784 -0.670755  0.169685  0.667724   
9  1941.0 -3.999320e-16  1.000258 -4.987308 -0.331460 -0.170913  0.524788   

         max  
0   2.178176  
1   2.201080  
2   6.390192  
3   6.390138  
4  29.176967  
5  34.327667  
6  42.378466  
7  22.2

In [3]:
optimizers = ["adam", "sgd", "rmsprop"]
activation = ["tanh", "sigmoid", "relu"]
input = 27
output = 34
hiddenLayers = []
lr = 0.001
batchSize = int(1941*0.8)
maxEpochs = 1000
seed = 42